# Hugging Face

## Step 1: Install Required Libraries
Install the Hugging Face libraries:

In [ ]:
pip install transformers datasets torch
pip install huggingface_hub


**Format your data as text pairs:**
Hugging Face provides numerous lightweight models that you can fine-tune. For summarization or analysis, you might use:

**Format your data as text pairs:**
BART (Sequence-to-Sequence)
DistilBERT (Compact version of BERT)
Example: Choosing a summarization model like T5-small for compactness.

**Format your data as text pairs:**
If you have a custom dataset:

**Format your data as text pairs:**
Input: Large text (to summarize or analyze)
Output: Summary or analysis results.
Save the dataset in .csv or .json.
Example format:

In [ ]:
# csv
input_text,summary_text
"This is a long article about AI...", "AI revolutionizes industries."


In [ ]:
# Alternatively, use a public dataset like CNN/DailyMail:

from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")


## **Step 4: Fine-tune the Model**
Use the Trainer API for fine-tuning

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

# Load the tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use a small portion for testing

# Tokenize data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()


## Step 5: Save the Fine-tuned Model
Save the model for later use:

In [ ]:
model.save_pretrained("./my-mini-model")
tokenizer.save_pretrained("./my-mini-model")


## Step 6: Test the Mini Model
Generate a summary or perform analysis using your fine-tuned mode

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the saved model
model = T5ForConditionalGeneration.from_pretrained("./my-mini-model")
tokenizer = T5Tokenizer.from_pretrained("./my-mini-model")

# Input text
text = "This is a long article about the importance of AI in modern industries."

# Generate a summary
input_ids = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).input_ids
outputs = model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0]))


## Step 7: Share on Hugging Face
Create an account on Hugging Face.
Use the huggingface_hub library to push your model

In [ ]:
from huggingface_hub import login

login(token="your_hugging_face_token")

model.push_to_hub("my-mini-model")
tokenizer.push_to_hub("my-mini-model")


# **Now your model is hosted and ready to use**